<a href="https://colab.research.google.com/github/OseiasBeu/Cardiovascular-diseases-risk-project/blob/main/ML_Cardiovascular_diseases_risk_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libs

In [1]:
#Leitura do dataset
import requests
import zipfile
import io
import os
import numpy as np
import pandas as pd

#Modelos
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier

#Métricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Exportação do Modelo
import joblib
import pickle

## Carregamento dos datasets

In [3]:
import requests
import os

github_url = 'https://api.github.com/repos/OseiasBeu/Cardiovascular-diseases-risk-project/contents/dataset.zip'

response = requests.get(github_url)


if response.status_code == 200:
    file_info = response.json()

    download_url = file_info.get('download_url')

    if download_url:
        response = requests.get(download_url)
        if response.status_code == 200:
            file_path = 'dataset.zip'

            with open(file_path, 'wb') as file:
                file.write(response.content)

            print(f'O arquivo foi baixado com sucesso e salvo em: {file_path}')
        else:
            print(f'Erro ao fazer o download do arquivo. Status Code: {response.status_code}')
    else:
        print('Não foi possível obter o URL de download do arquivo.')
else:
    print(f'Erro ao acessar a URL do GitHub. Status Code: {response.status_code}')


O arquivo foi baixado com sucesso e salvo em: dataset.zip


In [4]:
import zipfile

# Especifique o caminho para o arquivo ZIP baixado
zip_file_path = 'dataset.zip'

# Especifique o diretório de destino para a extração
extracted_dir_path = 'dataset/'

# Verifique se o arquivo ZIP existe
if not zipfile.is_zipfile(zip_file_path):
    print(f'O arquivo {zip_file_path} não é um arquivo ZIP válido.')
else:
    # Crie o diretório de destino se ele não existir
    if not os.path.exists(extracted_dir_path):
        os.makedirs(extracted_dir_path)

    # Abra o arquivo ZIP para extração
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extraia todo o conteúdo para o diretório de destino
        zip_ref.extractall(extracted_dir_path)

    print(f'O arquivo ZIP foi extraído com sucesso para o diretório: {extracted_dir_path}')


O arquivo ZIP foi extraído com sucesso para o diretório: dataset/


In [13]:
X_train_r = pd.read_csv('/content/dataset/dataset/X_train_r_cdr.csv',sep=';')
y_train_r = pd.read_csv('/content/dataset/dataset/y_train_r_cdr.csv',sep=';')

X_test_r = pd.read_csv('/content/dataset/dataset/X_test_r_cdr.csv',sep=';')
y_test_r = pd.read_csv('/content/dataset/dataset/y_teste_r_cdr.csv',sep=';')

X_train = pd.read_csv('/content/dataset/dataset/X_train_cdr.csv',sep=';')
y_train = pd.read_csv('/content/dataset/dataset/y_train_cdr.csv',sep=';')

X_test = pd.read_csv('/content/dataset/dataset/X_test_cdr.csv',sep=';')
y_test = pd.read_csv('/content/dataset/dataset/y_teste_cdr.csv',sep=';')

In [14]:
X_train_r.drop(columns=['Unnamed: 0'], inplace=True)
y_train_r.drop(columns=['Unnamed: 0'], inplace=True)

X_test_r.drop(columns=['Unnamed: 0'], inplace=True)
y_test_r.drop(columns=['Unnamed: 0'], inplace=True)

X_train.drop(columns=['Unnamed: 0'], inplace=True)
y_train.drop(columns=['Unnamed: 0'], inplace=True)

X_test.drop(columns=['Unnamed: 0'], inplace=True)
y_test.drop(columns=['Unnamed: 0'], inplace=True)

## Linear SVC

### Dados Balanceados

In [25]:
modelo = LinearSVC()
modelo.fit(X_train_r, y_train_r['Heart_Disease'])
previsoes = modelo.predict(X_test_r)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


#### Métricas

- Acurácia

In [26]:
print('Acurácia')
taxa_de_acerto = accuracy_score(y_test_r['Heart_Disease'], previsoes)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

print(10*'=====')

print('Precisão')
print("Precisão: %.20f" %(precision_score(y_test_r['Heart_Disease'], previsoes)))

print(10*'=====')
print('Recall')
print(recall_score(y_test_r['Heart_Disease'], previsoes))

print(10*'=====')
print('Matriz de Confusão:')
print(confusion_matrix(y_test_r['Heart_Disease'], previsoes))

Acurácia
Taxa de acerto 50.69
Precisão
Precisão: 0.91705069124423965743
Recall
0.017488739975832144
Matriz de Confusão:
[[45257    72]
 [44719   796]]


-

### Dados não balanceados

In [27]:
modelo = LinearSVC()
modelo.fit(X_train, y_train['Heart_Disease'])
previsoes = modelo.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


#### Métricas

In [28]:
print('Acurácia')
taxa_de_acerto = accuracy_score(y_test['Heart_Disease'], previsoes)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

print(10*'=====')

print('Precisão')
print("Precisão: %.2f" %(precision_score(y_test['Heart_Disease'], previsoes)))

print(10*'=====')
print('Recall')
print(recall_score(y_test['Heart_Disease'], previsoes))

print(10*'=====')
print('Matriz de Confusão:')
print(confusion_matrix(y_test['Heart_Disease'], previsoes))

Acurácia
Taxa de acerto 91.96
Precisão
Precisão: 0.59
Recall
0.017810686411847108
Matriz de Confusão:
[[56713    61]
 [ 4908    89]]


## BernoulliNB

In [34]:
bnb = BernoulliNB()

### Dados Balanceados

In [35]:
bnb.fit(X_train_r, y_train_r['Heart_Disease'])
predito_BNb = bnb.predict(X_test_r)

#### Métricas

In [36]:
print('Acurácia')
taxa_de_acerto = accuracy_score(y_test_r['Heart_Disease'], predito_BNb)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

print(10*'=====')

print('Precisão')
print("Precisão: %.20f" %(precision_score(y_test_r['Heart_Disease'], predito_BNb)))

print(10*'=====')
print('Recall')
print(recall_score(y_test_r['Heart_Disease'], predito_BNb))

print(10*'=====')
print('Matriz de Confusão:')
print(confusion_matrix(y_test_r['Heart_Disease'], predito_BNb))

Acurácia
Taxa de acerto 74.26
Precisão
Precisão: 0.73663836430908735675
Recall
0.756739536416566
Matriz de Confusão:
[[33015 12314]
 [11072 34443]]


### Não Balanceados

In [37]:
#treinar o modelo
bnb.fit(X_train, y_train['Heart_Disease'])
predito_BNb = bnb.predict(X_test)

taxa_de_acerto = accuracy_score(y_test['Heart_Disease'], predito_BNb)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

Taxa de acerto 86.68


#### Métricas

In [38]:
print('Acurácia')
taxa_de_acerto = accuracy_score(y_test['Heart_Disease'], predito_BNb)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

print(10*'=====')

print('Precisão')
print("Precisão: %.20f" %(precision_score(y_test['Heart_Disease'], predito_BNb)))

print(10*'=====')
print('Recall')
print(recall_score(y_test['Heart_Disease'], predito_BNb))

print(10*'=====')
print('Matriz de Confusão:')
print(confusion_matrix(y_test['Heart_Disease'], predito_BNb))

Acurácia
Taxa de acerto 86.68
Precisão
Precisão: 0.28160064891172098545
Recall
0.4168501100660396
Matriz de Confusão:
[[51460  5314]
 [ 2914  2083]]


## DTC

In [42]:
dtc = DecisionTreeClassifier(criterion='entropy', random_state=42)

### Dados Balanceados

In [43]:
dtc.fit(X_train_r, y_train_r['Heart_Disease'])
predito_ArvoreDecisao_r = dtc.predict(X_test_r)

#### Métricas

In [44]:
print('Acurácia')
taxa_de_acerto = accuracy_score(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

print(10*'=====')

print('Precisão')
print("Precisão: %.20f" %(precision_score(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r)))

print(10*'=====')
print('Recall')
print(recall_score(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r))

print(10*'=====')
print('Matriz de Confusão:')
print(confusion_matrix(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r))

Acurácia
Taxa de acerto 95.67
Precisão
Precisão: 0.92042467138523764714
Recall
1.0
Matriz de Confusão:
[[41394  3935]
 [    0 45515]]


Acurácia
Taxa de acerto 92.69
==================================================
Precisão
Precisão: 0.92355133262143962458
==================================================
Recall
0.9310996374821487
==================================================
Matriz de Confusão:
[[41821  3508]
 [ 3136 42379]]

### Não balanceados

In [20]:
dtc.fit(X_train, y_train['Heart_Disease'])
predito_ArvoreDecisao = dtc.predict(X_test)

#### Métricas

In [21]:
print('Acurácia')
taxa_de_acerto = accuracy_score(y_test['Heart_Disease'], predito_ArvoreDecisao)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

print(10*'=====')

print('Precisão')
print("Precisão: %.20f" %(precision_score(y_test['Heart_Disease'], predito_ArvoreDecisao)))

print(10*'=====')
print('Recall')
print(recall_score(y_test['Heart_Disease'], predito_ArvoreDecisao))

print(10*'=====')
print('Matriz de Confusão:')
print(confusion_matrix(y_test['Heart_Disease'], predito_ArvoreDecisao))

Acurácia
Taxa de acerto 86.77
Precisão
Precisão: 0.20672455200443376477
Recall
0.22393436061636982
Matriz de Confusão:
[[52480  4294]
 [ 3878  1119]]


## Escolha do modelos

1. **Acurácia:** Percentual e vezes em que o modelo acertou. Embora seja fácil de entender, a acurácia pode não ser a melhor métrica em todos os cenários, especialmente quando as classes estão desbalanceadas.

2. **Recall (Sensibilidade ou Taxa de Verdadeiros Positivos):** O recall mede a proporção de exemplos positivos reais que foram corretamente classificados como positivos pelo modelo. É sobre pegar tudo o que é positivo. Quanto do "positivo" real o modelo capturou?

3. **Precisão:** Em outras palavras, é a capacidade do modelo de não classificar erroneamente exemplos negativos como positivos. É sobre não se enganar quando diz que é positivo. Quanto do que o modelo diz ser positivo realmente é positivo? Exemplo: Dizer que uma moeda é justa e ela realmente ser justa.

4. **Matriz de Confusão:** A matriz de confusão é uma tabela que apresenta um resumo das classificações feitas pelo modelo, comparando essas classificações com os resultados reais. Ela é especialmente útil para entender o desempenho do modelo em diferentes classes. A matriz de confusão é composta por quatro valores:
  - Verdadeiros Positivos (VP): Exemplos positivos corretamente classificados como positivos.
  - Verdadeiros Negativos (VN): Exemplos negativos corretamente classificados como negativos.
  - Falsos Positivos (FP): Exemplos negativos erroneamente classificados como positivos.
  - Falsos Negativos (FN): Exemplos positivos erroneamente classificados como negativos.

Em resumo:
Pense em um teste de bola de futebol:
- **Acurácia:** Quantas vezes o jogador acertou os chutes?
- **Recall:** Quantas vezes o jogador pegou a bola quando ela estava vindo em sua direção?
- **Precisão:** Quantas vezes o jogador disse que ia marcar um gol e realmente marcou?
- **Matriz de Confusão:** Quantas vezes o jogador achou que a bola iria para a esquerda, mas foi para a direita, e vice-versa?

In [ ]:
print('Acurácia')
taxa_de_acerto = accuracy_score(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

print(10*'=====')

print('Precisão')
print("Precisão: %.20f" %(precision_score(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r)))

print(10*'=====')
print('Recall')
print(recall_score(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r))

print(10*'=====')
print('Matriz de Confusão:')
print(confusion_matrix(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r))

Acurácia
Taxa de acerto 92.69
Precisão
Precisão: 0.92355133262143962458
Recall
0.9310996374821487
Matriz de Confusão:
[[41821  3508]
 [ 3136 42379]]


  - Tendo como base as métricas listadas e explicadas anteriosmente, o melhor modelo para este caso é o modelo de `Árvore de decisão` com `dados balanceados`

## Exportando o modelo

In [45]:
filename = 'dtc_model_cdr_r.pkl'

with open(filename, 'wb') as file:
    pickle.dump(dtc, file)